In [14]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import os
import logging
load_dotenv()

False

In [15]:
df = pd.read_csv('/Users/anastasiahimic/hse_python_project_2-1/data_scrapping/files/genius.csv')

In [19]:
client_id = os.getenv('SPOTIPY_CLIENT_ID') or os.getenv('client_id_sp')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET') or os.getenv('client_secret_sp')
try:
    auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(auth_manager=auth_manager)
    logging.info("Аутентификация (Client Credentials) успешно выполнена")
except Exception as e:
    logging.error(f"Ошибка при аутентификации (Client Credentials): {e}")
    raise

ERROR:root:Ошибка при аутентификации (Client Credentials): No client_id. Pass it or set a SPOTIPY_CLIENT_ID environment variable.


SpotifyOauthError: No client_id. Pass it or set a SPOTIPY_CLIENT_ID environment variable.

In [9]:
def get_track_genres(track_name, artist_name=''):
    try:
        query = f'track:{track_name}'
        if artist_name:
            query += f' artist:{artist_name}'
        
        results = sp.search(q=query, type='track', limit=1)
        
        if results['tracks']['items']:
            artist_id = results['tracks']['items'][0]['artists'][0]['id']
            artist_info = sp.artist(artist_id)
            genres = artist_info['genres']
            return ', '.join(genres) if genres else 'No genres'
        else:
            return 'Track not found'
    except Exception as e:
        return f'Error: {str(e)}'

# Применить к DataFrame
df['genres'] = df['title'].apply(lambda x: get_track_genres(x))
    

In [10]:
print(df[['title', 'genres']].head())

                       title                           genres
0                 God’s Plan  Error: name 'sp' is not defined
1             In My Feelings  Error: name 'sp' is not defined
2              Hotline Bling  Error: name 'sp' is not defined
3                  One Dance  Error: name 'sp' is not defined
4  Hold On, We’re Going Home  Error: name 'sp' is not defined
